In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall tensorflow tensorboard tensorflow-text tf-keras ml-dtypes -y
!pip install tensorflow==2.15.0 tensorflow-addons==0.21.0

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

print("TensorFlow Version:", tf.__version__)
print("TensorFlow Addons Version:", tfa.__version__)

In [ ]:
!pip install mplfinance
!pip install Pillow
!pip install seaborn

In [ ]:
import mplfinance as fplt
import matplotlib
matplotlib.use('agg')
from matplotlib import image
from matplotlib import pyplot
import pandas as pd
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import seaborn as sn
from sklearn.metrics import classification_report,accuracy_score

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
forex_df_data = pd.read_csv('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/Datasets/MarketMovement/EURUSD/Seqs_2000_1_3_0.txt', sep=";")
forex_df_labels = pd.read_csv('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/Datasets/MarketMovement/EURUSD/Labels_2000_1_3_0.txt', sep=";")
forex_df = pd.merge(forex_df_data, forex_df_labels, on='IDTime')
forex_df.rename(columns={'ReadableTime': 'Date'}, inplace=True)
forex_df['Date']= pd.to_datetime(forex_df['Date']) 
forex_df['Open'] = forex_df['Close']
forex_df['Open'] = forex_df['Open'].shift(1)
forex_df.head(5)

In [ ]:
forex_ohlc = forex_df[["Date","Open","High","Low","Close","Class"]].copy()
dt_range = pd.date_range(start="2010.01.01", end="2020.01.01", freq='240min')
forex_ohlc = forex_ohlc[forex_ohlc.Date.isin(dt_range)]
forex_ohlc.reset_index(drop=True,inplace=True)
forex_ohlc.head(5)
total_size = forex_ohlc.shape[0]
train_size = int(0.8*total_size)
test_size = total_size - train_size
min_val = forex_ohlc.Low.min()
max_val = forex_ohlc.High.max()
print("Train set size = {0} and Test set size = {1}".format(train_size,test_size))
train_df = forex_ohlc.iloc[:train_size,:]
test_df = forex_ohlc.iloc[train_size:,:]

train_df

print("Training Dataset\n", train_df['Class'].value_counts(dropna=False))
print("Test Dataset\n", test_df['Class'].value_counts(dropna=False))

print(train_df.head(5))
print(train_df.tail(5))
print(test_df.head(5))
print(test_df.tail(5))
print(min_val, max_val)

In [ ]:
train_df.set_index('Date', inplace=True)
test_df.set_index('Date', inplace=True)

In [ ]:
from random import random, seed
seed(42)
def generate_image(sample_df, save_location_template, fname, sample_stay=False):
  class_name = sample_df.iloc[-1,4] # class of the last operation in the sequence
  roll = 1
  if sample_stay and class_name == 'stay':
    roll = random()
  if roll > 0.8: # only keep 20% of the stay samples because we have so many
    fplt.plot(
        sample_df,
        type='candle',
        style = "yahoo",
      figratio=(1,1),axisoff=True,tight_layout = True,mav=(2,4,6),
        savefig=dict(fname=save_location_template.format(class_name, fname),dpi=100),
        #ylim=(min_val,max_val)
    )

In [ ]:
import glob
from random import shuffle
matplotlib.use('module://ipykernel.pylab.backend_inline')
buy_fnames = glob.glob('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Train/buy/*.png')
sell_fnames = glob.glob('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Train/sell/*.png')
stay_fnames = glob.glob('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Train/stay/*.png')
shuffle(buy_fnames)
shuffle(sell_fnames)
shuffle(stay_fnames)

ROWS = 4
COLS = 4

fig, axes = plt.subplots(ROWS, COLS)
fig.suptitle('BUY')
for r in range(ROWS):
  for c in range(COLS):
    im = plt.imread(buy_fnames[r*ROWS+c])
    axes[r][c].set_axis_off()
    axes[r][c].patch.set_edgecolor('black')  
    axes[r][c].patch.set_linewidth('1')  
    axes[r][c].imshow(im)
    

fig, axes = plt.subplots(ROWS, COLS)
fig.suptitle('STAY')
for r in range(ROWS):
  for c in range(COLS):
    im = plt.imread(stay_fnames[r*ROWS+c])
    axes[r][c].set_axis_off()
    axes[r][c].patch.set_edgecolor('black')  
    axes[r][c].patch.set_linewidth('1')  
    axes[r][c].imshow(im)

fig, axes = plt.subplots(ROWS, COLS)
fig.suptitle('SELL')
for r in range(ROWS):
  for c in range(COLS):
    im = plt.imread(sell_fnames[r*ROWS+c])
    axes[r][c].set_axis_off()
    axes[r][c].patch.set_edgecolor('black')  
    axes[r][c].patch.set_linewidth('1')  
    axes[r][c].imshow(im)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
  for units in hidden_units:
    x = layers.Dense(units, activation=tf.nn.gelu)(x)
    x = layers.Dropout(dropout_rate)(x)
  return x

class Patches(layers.Layer):
  def __init__(self, patch_size):
    super(Patches, self).__init__()
    self.patch_size = patch_size

  def call(self, images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
      images=images,
      sizes=[1, self.patch_size, self.patch_size, 1],
      strides=[1, self.patch_size, self.patch_size, 1],
      rates=[1, 1, 1, 1],
      padding="VALID",
    )
    patch_dims = patches.shape[-1]
    patches = tf.reshape(patches, [batch_size, -1, patch_dims])
    return patches
  
class PatchEncoder(layers.Layer):
  def __init__(self, num_patches, projection_dim):
    super(PatchEncoder, self).__init__()
    self.num_patches = num_patches
    self.projection = layers.Dense(units=projection_dim)
    self.position_embedding = layers.Embedding(
      input_dim=num_patches, output_dim=projection_dim
    )

  def call(self, patch):
    positions = tf.range(start=0, limit=self.num_patches, delta=1)
    encoded = self.projection(patch) + self.position_embedding(positions)
    return encoded

def create_vit_classifier(input_shape, data_preprocessing):
  inputs = layers.Input(shape=input_shape)
  # Augment data.
  augmented = data_preprocessing(inputs)
  # Create patches.
  patches = Patches(patch_size)(augmented)
  # Encode patches.
  encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

  # Create multiple layers of the Transformer block.
  for _ in range(transformer_layers):
    # Layer normalization 1.
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    # Create a multi-head attention layer.
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=projection_dim, dropout=0.1
    )(x1, x1)
    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])
    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    # MLP.
    x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    # Skip connection 2.
    encoded_patches = layers.Add()([x3, x2])

  # Create a [batch_size, projection_dim] tensor.
  representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
  representation = layers.Flatten()(representation)
  representation = layers.Dropout(0.5)(representation)
  # Add MLP.
  features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
  # Classify outputs.
  logits = layers.Dense(3)(features)
  #logits = layers.Dense(1, activation='tanh')(features)
  # Create the Keras model.
  model = keras.Model(inputs=inputs, outputs=logits)
  return model

In [ ]:
## Test patch generation
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt

IMAGE_SIZE=224
PATCH_SIZE=16

plt.figure(figsize=(4, 4))
image = np.asarray(plt.imread('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Train/buy/yahoo_10007.png'))[...,:3]
plt.imshow(image)
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(IMAGE_SIZE, IMAGE_SIZE)
)
patches = Patches(PATCH_SIZE)(resized_image)
print(f"Image size: {IMAGE_SIZE} X {IMAGE_SIZE}")
print(f"Patch size: {PATCH_SIZE} X {PATCH_SIZE}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4), facecolor=(0,0,0))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (PATCH_SIZE, PATCH_SIZE, 3))
    plt.imshow(patch_img.numpy())
    plt.axis("off")

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 65
image_size = 224  # We'll resize input images to this size
patch_size = 16   # Size of the patches to be extracted from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
import glob
from PIL import Image

LBL_MAP = {'buy':0,'sell':1,'stay':2}

train_fnames = glob.glob('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Train/*/*.png')
val_fnames = glob.glob('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Val/*/*.png')
test_fnames = glob.glob('/kaggle/input/fortrademl/AI_forex_fisichella_garolla_master/content/Test/*/*.png')

x_train = np.asarray([np.asarray(Image.open(fn).convert('RGB').resize((image_size, image_size))) for fn in tqdm(train_fnames)])
y_train = keras.utils.to_categorical(np.asarray([LBL_MAP[fn.split('/')[-2]] for fn in train_fnames]), num_classes=3)

x_val   = np.asarray([np.asarray(Image.open(fn).convert('RGB').resize((image_size, image_size))) for fn in tqdm(val_fnames)])
y_val   = keras.utils.to_categorical(np.asarray([LBL_MAP[fn.split('/')[-2]] for fn in val_fnames]), num_classes=3)

x_test   = np.asarray([np.asarray(Image.open(fn).convert('RGB').resize((image_size, image_size))) for fn in tqdm(test_fnames)])
y_test   = keras.utils.to_categorical(np.asarray([LBL_MAP[fn.split('/')[-2]] for fn in test_fnames]), num_classes=3)


data_preprocessing = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization()
    ],
    name="data_preprocessing",
)
# Compute the mean and the variance of the training data for normalization.
data_preprocessing.layers[0].adapt(x_train)

In [ ]:
from sklearn.metrics import confusion_matrix

def run_experiment(model, x_train, y_train, x_val, y_val, x_test, y_test, model_name):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
        ],
    )

    checkpoint_filepath = f"/kaggle/working/{model_name}/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    class_weights = {
    0: 3.0,  # 'Stay' (more weight due to low samples)
    1: 3.0,  # 'Buy'
    2: 2.0   # 'Sell' (reduce weight)
    }
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(x_val,y_val),
        validation_batch_size=batch_size,
        callbacks=[checkpoint_callback],
        shuffle=True,
        class_weight=class_weights
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(x_train, y_train)
    print('\nPerformance of best model:')
    print(f"> Train accuracy: {accuracy}")
    _, accuracy = model.evaluate(x_val, y_val)
    print(f"> Val accuracy: {accuracy}")
    _, accuracy = model.evaluate(x_test, y_test)
    print(f"> Test accuracy: {accuracy}")

    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    print('Confusion Matrix:')
    print(cm)

    return history


# ViT

In [ ]:
vit_classifier = create_vit_classifier((image_size, image_size, 3),data_preprocessing)
history = run_experiment(vit_classifier, x_train, y_train, x_val, y_val, x_test, y_test, 'ViT_224')

In [ ]:
# Saving the model
vit_classifier.save("vit_model")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# 1. Predict probabilities and take argmax to get the class predictions
y_pred = vit_classifier.predict(x_test)  # Predict
y_pred_classes = y_pred.argmax(axis=1)   # Convert probabilities to class labels
y_true_classes = y_test.argmax(axis=1)   # True class labels

# 2. Define class labels
class_labels = ['Stay', 'Buy', 'Sell']  # Custom labels for your classes

# 3. Generate confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# 4. Display the confusion matrix with labels
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Buy, Sell, Stay")
plt.show()

# 5. Print detailed classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_labels))

# ResNet50

In [ ]:
x_train_resnet = tf.keras.applications.resnet.preprocess_input(x_train)
x_val_resnet = tf.keras.applications.resnet.preprocess_input(x_val)
x_test_resnet = tf.keras.applications.resnet.preprocess_input(x_test)

In [ ]:
# # Delete the non-resnet x data to save memory
# del x_train
# del x_val
# del x_test

In [ ]:
resnet_classifier = tf.keras.models.Sequential([
  tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3)),
  layers.GlobalAveragePooling2D(name='avg_pool'),
  layers.Dense(3, name='logits')
])

In [ ]:
history = run_experiment(resnet_classifier, x_train_resnet, y_train, x_val_resnet, y_val, x_test_resnet, y_test, 'ResNet50')
resnet_classifier.save("ResNet50")

# Performing Undersampling

In [ ]:
# Function to perform undersampling
def undersample_data(x, y, target_class, target_count):
    indices = np.where(y.argmax(axis=1) == target_class)[0]
    np.random.shuffle(indices)
    selected_indices = indices[:target_count]
    return x[selected_indices], y[selected_indices]

# Determine the target count for all classes (equal to the minority class count)
target_count = min(np.sum(y_train.argmax(axis=1) == i) for i in range(3))

# Undersample the SELL class (class 2)
x_train_sell, y_train_sell = undersample_data(x_train, y_train, target_class=2, target_count=target_count)

# Keep all samples of the BUY and STAY classes
x_train_buy = x_train[y_train.argmax(axis=1) == 1]
y_train_buy = y_train[y_train.argmax(axis=1) == 1]
x_train_stay = x_train[y_train.argmax(axis=1) == 0]
y_train_stay = y_train[y_train.argmax(axis=1) == 0]

# Combine the undersampled and original data to form the balanced dataset
x_train_balanced = np.concatenate([x_train_buy, x_train_sell, x_train_stay], axis=0)
y_train_balanced = np.concatenate([y_train_buy, y_train_sell, y_train_stay], axis=0)

# Shuffle the balanced training dataset
indices = np.arange(x_train_balanced.shape[0])
np.random.shuffle(indices)
x_train_balanced = x_train_balanced[indices]
y_train_balanced = y_train_balanced[indices]

In [ ]:
# Analyze the balanced class distribution
print("Balanced Class Distribution:")
print(pd.DataFrame({'Class': [0, 1, 2], 'Count': np.sum(y_train_balanced, axis=0)}))

# ViT

In [ ]:
vit_classifier = create_vit_classifier((image_size, image_size, 3), data_preprocessing)
history_vit = run_experiment(
    vit_classifier,
    x_train_balanced,
    y_train_balanced,
    x_val,
    y_val,
    x_test,
    y_test,
    "ViT_224_Balanced",
)

In [ ]:
vit_classifier.save("vit_model_balanced")

# Resnet50

In [ ]:
# Prepare ResNet50 inputs
x_train_resnet_balanced = tf.keras.applications.resnet.preprocess_input(x_train_balanced)
x_val_resnet = tf.keras.applications.resnet.preprocess_input(x_val)
x_test_resnet = tf.keras.applications.resnet.preprocess_input(x_test)

# Train the ResNet50 model
resnet_classifier = tf.keras.models.Sequential([
    tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3)),
    layers.GlobalAveragePooling2D(name="avg_pool"),
    layers.Dense(3, name="logits"),
])
history_resnet = run_experiment(
    resnet_classifier,
    x_train_resnet_balanced,
    y_train_balanced,
    x_val_resnet,
    y_val,
    x_test_resnet,
    y_test,
    "ResNet50_Balanced",
)

In [ ]:
resnet_classifier.save("ResNet50_balanced")

# Piplining

In [ ]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# import glob
# from PIL import Image
# import mplfinance as fplt
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import LabelEncoder

In [ ]:
# model = tf.keras.models.load_model('/kaggle/working/vit_model')

## Load data

In [ ]:
# new_data = pd.read_csv('/kaggle/input/new-data-m1/data')

In [ ]:
# import os
# import pandas as pd
# import mplfinance as fplt

# import pandas as pd

# def preprocess_new_data(df):
#     # Check and rename columns to match required format
#     required_columns = ['time', 'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c']
#     if not all(col in df.columns for col in required_columns):
#         raise KeyError(f"Missing required columns: {set(required_columns) - set(df.columns)}")
    
#     # Create 'Date' column from 'time' (ensure datetime format)
#     df['Date'] = pd.to_datetime(df['time'])
    
#     # Create OHLC columns (average of bid and ask prices)
#     df['Open'] = (df['bid_o'] + df['ask_o']) / 2
#     df['High'] = (df['bid_h'] + df['ask_h']) / 2
#     df['Low'] = (df['bid_l'] + df['ask_l']) / 2
#     df['Close'] = (df['bid_c'] + df['ask_c']) / 2

#     # Select only the required columns for candlestick plotting
#     df = df[['Date', 'Open', 'High', 'Low', 'Close']]

#     # Set 'Date' as the index
#     df.set_index('Date', inplace=True)

#     return df



# def generate_candlestick_images(df, save_image_path):
#     # Ensure the save path exists
#     if not os.path.exists(save_image_path):
#         os.makedirs(save_image_path)
    
#     for idx in range(len(df)):
#         # Select the last 50 rows (candles)
#         candle_data = df.iloc[max(0, idx-49):idx+1]
        
#         # Generate and save the candlestick chart
#         save_file = os.path.join(save_image_path, f"candle_{idx}.png")
#         fplt.plot(
#             candle_data,
#             type='candle',
#             style='yahoo',
#             savefig=dict(fname=save_file, dpi=100)
#         )

In [ ]:
# import cv2
# import numpy as np

# def predict_actions(image_folder):
#     predictions = []
#     image_files = sorted(os.listdir(image_folder))  # Sort by file name for consistency
    
#     for image_file in image_files:
#         image_path = os.path.join(image_folder, image_file)
        
#         # Load the image and preprocess it for the model
#         image = cv2.imread(image_path)
#         image = cv2.resize(image, (224, 224))  # Assuming model expects 224x224 input
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
#         image = image / 255.0  # Normalize pixel values
#         image = np.expand_dims(image, axis=0)  # Add batch dimension
        
#         # Predict the action
#         prediction = model.predict(image)
#         action = np.argmax(prediction)  # Get the index of the highest probability
        
#         # Map the index to the action
#         action_map = {0: "stay", 1: "buy", 2: "sell"}
#         predictions.append(action_map[action])
    
#     return predictions

In [ ]:
# # Preprocess the data
# preprocessed_data = preprocess_new_data(new_data)

# # Generate candlestick images
# generate_candlestick_images(preprocessed_data, save_image_path="/kaggle/working/images")

In [ ]:
# predictions = predict_actions("/kaggle/working/images")
# print("Predicted Actions:", predictions)